In [1]:
# Test voice recognition, call player, voice over

import os, re
import torch; torch._dynamo.config.recompile_limit = 64;

In [2]:
# Грузим модель, которая распознает голос и создает запрос.
from unsloth import FastModel


model, processor = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it",
    dtype = None,
    max_seq_length = 1024,
    load_in_4bit = True,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/m1/nvme/p311uns2/lib/python3.11/site-packages/unsloth_zoo/gradient_checkpointing.py:355: UserWarning: expandable_segments not supported on this platform (Triggered internally at /pytorch/c10/hip/HIPAllocatorConfig.h:36.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2026.1.4: Fast Gemma3N patching. Transformers: 4.57.6.
   \\   /|    Radeon RX 7900 XT. Num GPUs = 1. Max memory: 19.984 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+rocm6.4. ROCm Toolkit: 6.4.43482-0f2d60242. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(f"Attn Implementation: {model.config._attn_implementation}")

Attn Implementation: sdpa


In [4]:
# функции для инференса

from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template


# Check out available chat templates and how they are applied
# https://github.com/unslothai/unsloth/blob/main/unsloth/chat_templates.py
tokenizer = get_chat_template(processor, chat_template="gemma3n")

# Helper function for inference
def streaming_inference(messages, max_new_tokens = 128):
    _ = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True,
            tokenize = True,
            return_dict = True,
            return_tensors = "pt",
        ).to("cuda"),
        max_new_tokens = max_new_tokens,
        do_sample=False,
        streamer = TextStreamer(processor, skip_prompt = True),
    )

In [5]:
# Класс  функция для получения транскрибации из голоса с одновременным выводом на экран

from transformers import TextStreamer
import io
import sys


class CaptureTextStreamer(TextStreamer):
    def __init__(self, tokenizer, skip_prompt=True, **kwargs):
        super().__init__(tokenizer, skip_prompt=skip_prompt, **kwargs)
        self.generated_text = ""
        
    def on_finalized_text(self, text: str, stream_end: bool = False):
        # Сохраняем текст
        self.generated_text += text
        # Также выводим в консоль (опционально)
        print(text, end="", flush=True)

# Модифицированная функция для захвата текста
def capture_inference(messages, max_new_tokens=128):
    # Создаем кастомный streamer
    streamer = CaptureTextStreamer(processor, skip_prompt=True)
    
    # Генерируем ответ
    output = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to("cuda"),
        max_new_tokens=max_new_tokens,
        do_sample=False,
        streamer=streamer,
    )
    
    # Возвращаем захваченный текст
    return streamer.generated_text

In [6]:
from pathlib import Path

test_audios = ['00101.wav','00102.wav']


In [7]:
# Класс audioplay для REST API управления плеером.

import csv
import json
import random
import re
from typing import Dict, List, Optional
import requests
import time
import sys

class audioplay:
    def __init__(self, host: str = "http://127.0.0.1:5000", csv_file: str = "music_base.csv"):
        """
        Инициализация класса. Считывает файл music_base.csv
        """
        self.host = host
        self.music_base = []
        try:
            with open(csv_file, 'r', encoding='utf-8-sig') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    self.music_base.append(row)
            print(f"Загружено {len(self.music_base)} записей из {csv_file}")
        except FileNotFoundError:
            print(f"Файл {csv_file} не найден. База данных пуста.")
        except Exception as e:
            print(f"Ошибка при чтении файла {csv_file}: {e}")
############################

    def find_in_base(self, artist: str = "", title: str = "", album: str = "", genre: str = "") -> Dict:
        """
        Поиск наиболее подходящей записи в базе данных
        
        Args:
            artist: Имя исполнителя для поиска
            title: Название трека для поиска
            album: Название альбома для поиска
            genre: Жанр для поиска
        
        Returns:
            Dict: Наиболее подходящая запись в виде словаря или False, если не найдено
        """
        if not self.music_base:
            return False
        
        # Создаем список критериев поиска (не пустых)
        search_criteria = []
        if artist:
            search_criteria.append(('Artist', artist))
        if title:
            search_criteria.append(('Title', title))
        if album:
            search_criteria.append(('Album', album))
        if genre:
            search_criteria.append(('Genre', genre))
        
        # Если нет критериев поиска, возвращаем False
        if not search_criteria:
            return False
        
        best_matches = []
        best_score = 0
        
        for record in self.music_base:
            score = 0
            for field, value in search_criteria:
                # Сравниваем без учета регистра
                if record.get(field, '').lower() == value.lower():
                    score += 1
            
            if score > 0:  # Только записи с хотя бы одним совпадением
                if score > best_score:
                    best_score = score
                    best_matches = [record]
                elif score == best_score:
                    best_matches.append(record)
        
        # Если найдены совпадения, выбираем случайное из лучших
        if best_matches:
            return random.choice(best_matches)
        else:
            # Если критерии есть, но нет совпадений, возвращаем False
            return False

###################
    
    def audio_play(self, playfile: Optional[Dict] = None):
        """
        Функция воспроизведения аудио
        
        Args:
            host: Хост для воспроизведения
            playfile: Словарь с данными о треке
        """
        if playfile:
            print(playfile)
            print(f"SourceFile = {playfile.get('SourceFile', 'N/A')}")
            SourceFile = playfile.get('SourceFile', 'N/A')
            print(f"FileName = {playfile.get('FileName', 'N/A')}")
            print(f"Artist = {playfile.get('Artist', 'N/A')}")
            print(f"Album = {playfile.get('Album', 'N/A')}")
            print(f"Title = {playfile.get('Title', 'N/A')}")
            print(f"Genre = {playfile.get('Genre', 'N/A')}")
            print(f"Воспроизведение на хосте (адрес:порт): {self.host}")
            ##########
            try:                
                
                resp = requests.post(
                    f"{self.host}/play",
                    json={"file_path": SourceFile},
                    headers={"Content-Type": "application/json"}
                )
                print(f"✅ Сервис доступен: {resp.json()}")
                return resp
            except:
                print("❌ Сервис не доступен!")
                print("Возможно надо запустить сначала сервис: python audio_player_service.py")
                return False
        else:
            print("Нет данных для воспроизведения")
            return False
    
    def audio_stop(self):
        """Функция остановки воспроизведения"""
        print("Play stopping")
        try:
            resp = requests.post(f"{self.host}/stop")
        except Exception as e:
            print("❌ Сервис не доступен!")
            print(e)
            return False
        return resp
        
    
    def audio_status(self):
        """Функция получения статуса воспроизведения"""
        print("Status of playing")
        try:
            resp = requests.get(f"{self.host}/status")
        except:
            print("❌ Сервис не доступен!")
            return False
        return resp
        
    
    def parse_xml_arguments(self, xml_content: str) -> Dict:
        """
        Парсинг аргументов из XML-строки
        
        Args:
            xml_content: XML-строка с аргументами
        
        Returns:
            Dict: Словарь с аргументами
        """
        args = {}
        
        # Извлекаем значения аргументов с помощью регулярных выражений
        artist_match = re.search(r'<artist>(.*?)</artist>', xml_content, re.DOTALL)
        title_match = re.search(r'<title>(.*?)</title>', xml_content, re.DOTALL)
        album_match = re.search(r'<album>(.*?)</album>', xml_content, re.DOTALL)
        genre_match = re.search(r'<genre>(.*?)</genre>', xml_content, re.DOTALL)
        
        if artist_match:
            args['artist'] = artist_match.group(1).strip()
        if title_match:
            args['title'] = title_match.group(1).strip()
        if album_match:
            args['album'] = album_match.group(1).strip()
        if genre_match:
            args['genre'] = genre_match.group(1).strip()
        
        return args
    
    def tool_call(self, xml_string: str) -> Dict:
        """
        Обработка XML-запроса
        
        Args:
            xml_string: XML-строка с вызовом инструмента
        
        Returns:
            Dict: Результат выполнения в формате JSON
        """
        # Удаляем все после </tool_call>
        if '</tool_call>' in xml_string:
            xml_string = xml_string.split('</tool_call>')[0] + '</tool_call>'
        
        # Извлекаем имя функции из XML
        name_match = re.search(r'<name>(.*?)</name>', xml_string)
        
        if not name_match:
            return {
                "status": "error",
                "message": "Invalid XML format: missing <name> tag"
            }
        
        function_name = name_match.group(1)
        
        # Проверяем, что вызов начинается с audioplay.
        if not function_name.startswith('audioplay.'):
            return {
                "status": "error",
                "message": "Don't run non audioplay tool."
            }
        
        # Извлекаем название метода
        method_name = function_name.split('.')[1] if '.' in function_name else ''
        
        # Обработка различных методов
        if method_name == 'play_request':
            # Парсим аргументы
            args = self.parse_xml_arguments(xml_string)
            
            # Ищем подходящую запись в базе
            found_record = self.find_in_base(
                artist=args.get('artist', ''),
                title=args.get('title', ''),
                album=args.get('album', ''),
                genre=args.get('genre', '')
            )
            
            if found_record:
                # Вызываем функцию воспроизведения
                if self.audio_play(playfile=found_record):
                    # Возвращаем найденную запись
                    return {
                        "status": "success",
                        "data": found_record
                    }
                else: # Запустить не удалось
                    return {
                        "status": "error",
                        "message": "Can't start playing"
                    }
            else: # Нет подходящей записи
                return {
                    "status": "error",
                    "message": "No matching records found"
                }
        
        elif method_name == 'play_stop':
            self.audio_stop()
            return {
                "status": "success",
                "message": "Playback stopped"
            }
        
        elif method_name == 'play_status':
            self.audio_status()
            return {
                "status": "success",
                "message": "Status requested"
            }
        
        else:
            return {
                "status": "error",
                "message": "Unknown audioplay method"
            }


In [8]:
import re
import requests
from typing import Dict, Optional

class speech_text:
    def __init__(self, host: str = "http://127.0.0.1:5001", speaker: str = 'ls_female1.wav'):
        """
        Инициализация класса для взаимодействия с TTS сервисом.
        
        Args:
            host: Хост TTS сервиса
            speaker: Путь к файлу голоса по умолчанию
        """
        self.host = host
        self.default_speaker = speaker
        print(f"TTS сервис инициализирован: {host}, голос: {speaker}")

    def speak(self, text: str, speaker_wav: Optional[str] = None, language: Optional[str] = None) -> Dict:
        """
        Генерация и воспроизведение речи из текста.
        
        Args:
            text: Текст для озвучивания
            speaker_wav: Путь к файлу голоса (если None, используется default_speaker)
            language: Язык озвучки
            
        Returns:
            Dict: Результат выполнения
        """
        if not text:
            return {
                "status": "error",
                "message": "Не указан текст для озвучивания"
            }
        
        # Используем голос по умолчанию, если не указан
        if speaker_wav is None:
            speaker_wav = self.default_speaker
        
        try:
            print(f"Отправка текста на озвучку: '{text[:50]}...'")
            print(f"Используется голос: {speaker_wav}")
            
            resp = requests.post(
                f"{self.host}/tts/speak",
                json={
                    "text": str(text),
                    "speaker_wav": speaker_wav,
                    "language": language
                },
                headers={"Content-Type": "application/json"},
                timeout=10
            )
            
            if resp.status_code == 200:
                result = resp.json()
                print(f"✅ TTS сервис доступен: {result.get('message', 'Речь сгенерирована')}")
                return {
                    "status": "success",
                    "response": result
                }
            else:
                print(f"❌ Ошибка TTS сервиса: {resp.status_code}")
                try:
                    error_data = resp.json()
                    return {
                        "status": "error",
                        "message": error_data.get('message', f'Ошибка {resp.status_code}'),
                        "details": error_data
                    }
                except:
                    return {
                        "status": "error",
                        "message": f"Ошибка TTS сервиса: {resp.status_code}"
                    }
                    
        except requests.exceptions.ConnectionError:
            print("❌ TTS сервис не доступен!")
            print("Возможно надо запустить сначала сервис: python tts_service.py")
            return {
                "status": "error",
                "message": "TTS сервис не доступен"
            }
        except requests.exceptions.Timeout:
            print("❌ Таймаут соединения с TTS сервисом")
            return {
                "status": "error",
                "message": "Таймаут соединения с TTS сервисом"
            }
        except Exception as e:
            print(f"❌ Ошибка при вызове TTS сервиса: {e}")
            return {
                "status": "error",
                "message": str(e)
            }

    def generate_only(self, text: str, speaker_wav: Optional[str] = None, 
                     language: Optional[str] = None) -> Dict:
        """
        Только генерация аудиофайла без воспроизведения.
        
        Args:
            text: Текст для озвучивания
            speaker_wav: Путь к файлу голоса
            language: Язык озвучки
            
        Returns:
            Dict: Результат с путем к файлу или ошибка
        """
        if not text:
            return {
                "status": "error",
                "message": "Не указан текст для озвучивания"
            }
        
        if speaker_wav is None:
            speaker_wav = self.default_speaker
        
        try:
            print(f"Генерация аудиофайла для текста: '{text[:50]}...'")
            
            resp = requests.post(
                f"{self.host}/tts/generate",
                json={
                    "text": str(text),
                    "speaker_wav": speaker_wav,
                    "language": language
                },
                headers={"Content-Type": "application/json"},
                timeout=30
            )
            
            if resp.status_code == 200:
                # Если сервис возвращает файл
                if resp.headers.get('Content-Type', '').startswith('audio/'):
                    # Сохраняем файл
                    import tempfile
                    import os
                    
                    temp_file = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
                    temp_file.write(resp.content)
                    temp_file.close()
                    
                    print(f"✅ Аудиофайл сохранен: {temp_file.name}")
                    return {
                        "status": "success",
                        "file_path": temp_file.name,
                        "message": "Аудиофайл сгенерирован"
                    }
                else:
                    # Или возвращаем JSON ответ
                    result = resp.json()
                    return {
                        "status": "success",
                        "response": result
                    }
            else:
                return {
                    "status": "error",
                    "message": f"Ошибка генерации: {resp.status_code}"
                }
                
        except Exception as e:
            print(f"❌ Ошибка при генерации аудио: {e}")
            return {
                "status": "error",
                "message": str(e)
            }

    def stop(self) -> Dict:
        """
        Остановка текущего воспроизведения.
        
        Returns:
            Dict: Результат выполнения
        """
        try:
            print("Остановка воспроизведения TTS...")
            resp = requests.post(f"{self.host}/tts/stop")
            
            if resp.status_code == 200:
                result = resp.json()
                print(f"✅ {result.get('message', 'Воспроизведение остановлено')}")
                return {
                    "status": "success",
                    "response": result
                }
            else:
                return {
                    "status": "error",
                    "message": f"Ошибка остановки: {resp.status_code}"
                }
                
        except Exception as e:
            print(f"❌ Ошибка при остановке: {e}")
            return {
                "status": "error",
                "message": str(e)
            }

    def status(self) -> Dict:
        """
        Получение статуса TTS сервиса.
        
        Returns:
            Dict: Статус сервиса
        """
        try:
            print("Запрос статуса TTS сервиса...")
            resp = requests.get(f"{self.host}/tts/status")
            
            if resp.status_code == 200:
                result = resp.json()
                return {
                    "status": "success",
                    "response": result
                }
            else:
                return {
                    "status": "error",
                    "message": f"Ошибка запроса статуса: {resp.status_code}"
                }
                
        except Exception as e:
            print(f"❌ Ошибка при запросе статуса: {e}")
            return {
                "status": "error",
                "message": str(e)
            }

    def health_check(self) -> bool:
        """
        Проверка доступности TTS сервиса.
        
        Returns:
            bool: True если сервис доступен
        """
        try:
            resp = requests.get(f"{self.host}/tts/health", timeout=5)
            return resp.status_code == 200
        except:
            return False

    def tool_call(self, xml_string: str) -> Dict:
        """
        Обработка XML-запроса для TTS (если потребуется в будущем).
        Аналогично audioplay.tool_call, но для озвучки текста.
        
        Args:
            xml_string: XML-строка с вызовом инструмента
            
        Returns:
            Dict: Результат выполнения
        """
        # Удаляем все после </tool_call>
        if '</tool_call>' in xml_string:
            xml_string = xml_string.split('</tool_call>')[0] + '</tool_call>'
        
        # Извлекаем имя функции из XML
        name_match = re.search(r'<name>(.*?)</name>', xml_string)
        
        if not name_match:
            return {
                "status": "error",
                "message": "Invalid XML format: missing <name> tag"
            }
        
        function_name = name_match.group(1)
        
        # Проверяем, что вызов начинается с speech_text.
        if not function_name.startswith('speech_text.'):
            return {
                "status": "error",
                "message": "Don't run non speech_text tool."
            }
        
        # Извлекаем название метода
        method_name = function_name.split('.')[1] if '.' in function_name else ''
        
        # Парсим аргументы
        text_match = re.search(r'<text>(.*?)</text>', xml_string, re.DOTALL)
        speaker_match = re.search(r'<speaker>(.*?)</speaker>', xml_string, re.DOTALL)
        language_match = re.search(r'<language>(.*?)</language>', xml_string, re.DOTALL)
        
        args = {}
        if text_match:
            args['text'] = text_match.group(1).strip()
        if speaker_match:
            args['speaker'] = speaker_match.group(1).strip()
        if language_match:
            args['language'] = language_match.group(1).strip()
        
        # Обработка различных методов
        if method_name == 'speak':
            if 'text' in args:
                return self.speak(
                    text=args['text'],
                    speaker_wav=args.get('speaker'),
                    language=args.get('language')
                )
            else:
                return {
                    "status": "error",
                    "message": "Missing required argument: text"
                }
        
        elif method_name == 'generate':
            if 'text' in args:
                return self.generate_only(
                    text=args['text'],
                    speaker_wav=args.get('speaker'),
                    language=args.get('language')
                )
            else:
                return {
                    "status": "error",
                    "message": "Missing required argument: text"
                }
        
        elif method_name == 'stop':
            return self.stop()
        
        elif method_name == 'status':
            return self.status()
        
        else:
            return {
                "status": "error",
                "message": "Unknown speech_text method"
            }

In [9]:
spk = speech_text(host="http://127.0.0.1:5001", speaker='ls_female1.wav')


TTS сервис инициализирован: http://127.0.0.1:5001, голос: ls_female1.wav


In [10]:
player = audioplay(host="http://127.0.0.1:5000", csv_file = "music_base.csv")

Загружено 812 записей из music_base.csv


In [11]:
def transcribe_audio(filename):
    """ Распознает аудио из filename и выдает запрос для audio_play, если надо."""
    SYSTEM_PROMPT_V1 = """You are a tool-using assistant that can understand audio. You are given a user's request in audio format.
You MUST transcribe user's audio and respond with exactly one XML tool call if the tool usage is required (user is asking about play music, asking music title or artist or Genre or Album), otherwise respond to user's request in plain text.
Template: <tool_call><name>audioplay.play_request</name><arguments><artist>ARTIST</artist><title>TITLE</title><genre>GENRE</genre><album>Album</album></arguments></tool_call>. If some arguments is not recognised the correpondence field should be empty. If tool usage is not required respond shortly with text

Also You MUST transcribe user's control command's to stop playing, such as stop, break, cancel to following Template: <tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call>

Also You MUST transcribe user's control command's about playing status, such as status (give status) to following Template: <tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call>
"""

    messages = [            
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYSTEM_PROMPT_V1,
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {"type": "audio", "audio": filename},
                                
            ]
        }
    ]

    #print(messages)
    response = capture_inference(messages)
    return response
    

In [12]:
user_audio = test_audios[1] # Выбрали wav-файл
resp = transcribe_audio(user_audio) # транскрибация
print(f"\nresp = {resp}")

<tool_call><name>audioplay.play_request</name><arguments><artist>Duke Ellington</artist></arguments></tool_call><end_of_turn>
resp = <tool_call><name>audioplay.play_request</name><arguments><artist>Duke Ellington</artist></arguments></tool_call><end_of_turn>


In [13]:
tool_resp = player.tool_call(resp)
print(f"tool_resp = {tool_resp}")

{'SourceFile': '/home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3', 'FileName': 'Duke Ellington - Come sunday.mp3', 'Artist': 'Duke Ellington', 'Album': 'JazzmastersMPEG', 'Title': 'Come sunday', 'Genre': 'Jazz'}
SourceFile = /home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3
FileName = Duke Ellington - Come sunday.mp3
Artist = Duke Ellington
Album = JazzmastersMPEG
Title = Come sunday
Genre = Jazz
Воспроизведение на хосте (адрес:порт): http://127.0.0.1:5000
✅ Сервис доступен: {'file': '/home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3', 'message': 'Начато воспроизведение: Duke Ellington - Come sunday.mp3', 'status': 'success'}
tool_resp = {'status': 'success', 'data': {'SourceFile': '/home/m1/music1/Duke Ellington - JazzmastersMPEG/Duke Ellington - Come sunday.mp3', 'FileName': 'Duke Ellington - Come sunday.mp3', 'Artist': 'Duke Ellington', 'Album': 'JazzmastersMPEG', 'Title': 'Come sunday',

In [14]:
def text_for_tts(tool_resp):
    text_to_speak = "I don't know what to say"
    if isinstance(tool_resp, dict):
        # Формируем текст для озвучки на основе результата
        if tool_resp.get('status') == 'success':
            data = tool_resp.get('data', {})
            if data:
                # Если есть информация о треке
                artist = data.get('Artist', 'Unknown artist')
                title = data.get('Title', 'Unknown title')            
                text_to_speak = f"Playing '{title}' by  {artist}"
            else:
                text_to_speak = tool_resp.get('message', 'Success')
        else:
            text_to_speak = tool_resp.get('message', 'Error')

    return text_to_speak    

text_to_speak = text_for_tts(tool_resp)
print(f'text to speak = {text_to_speak}')
tts_resp = spk.speak(text_to_speak)
print(f"tts_resp = {tts_resp}")

text to speak = Playing 'Come sunday' by  Duke Ellington
Отправка текста на озвучку: 'Playing 'Come sunday' by  Duke Ellington...'
Используется голос: ls_female1.wav
✅ TTS сервис доступен: Начато воспроизведение сгенерированной речи
tts_resp = {'status': 'success', 'response': {'audio_samples': 76816, 'language': None, 'message': 'Начато воспроизведение сгенерированной речи', 'speaker': 'ls_female1.wav', 'status': 'success', 'text_length': 40}}


In [ ]:
# Это были примеры. Теперь запустим в цикле микрофон.

In [15]:
import wavio
import numpy as np
from silero_vad import load_silero_vad # Модель для endpoint detection (end of utterance)
import time
import sounddevice as sd

In [16]:
# Параметры аудио
SAMPLE_RATE = 16000
CHUNK_SIZE = 512  # Фиксированный размер для модели при 16 кГц
CHUNK_DURATION_MS = int((CHUNK_SIZE / SAMPLE_RATE) * 1000)

# Параметры детектирования конца реплики
PAUSE_THRESHOLD_MS = 500
PAUSE_CHUNKS = PAUSE_THRESHOLD_MS // CHUNK_DURATION_MS

# Загружаем модель и переводим в режим инференса
endpd_model = load_silero_vad()
endpd_model.eval()

# Состояние записи
audio_buffer = []
is_recording = False
pause_counter = 0

In [21]:
def callback(indata, frames, time_info, status):
    global audio_buffer, is_recording, pause_counter
    
    if status:
        print(f"Аудиостатус: {status}")
    
    # Конвертируем в моно и выравниваем размер
    if indata.ndim > 1:
        audio_chunk = indata.mean(axis=1).flatten()
    else:
        audio_chunk = indata.flatten()
    
    # Критически важно: размер чанка должен быть ровно 512 семплов
    if len(audio_chunk) > CHUNK_SIZE:
        audio_chunk = audio_chunk[:CHUNK_SIZE]
    elif len(audio_chunk) < CHUNK_SIZE:
        # Дополняем нулями (тишиной) до нужного размера
        padding = np.zeros(CHUNK_SIZE - len(audio_chunk))
        audio_chunk = np.concatenate([audio_chunk, padding])
    
    # Преобразуем в тензор PyTorch и получаем вероятность речи
    audio_tensor = torch.from_numpy(audio_chunk).float()
    
    with torch.no_grad():
        # Модель ожидает вход [batch_size, samples]
        # unsqueeze(0) добавляет размерность батча
        prob_tensor = endpd_model(audio_tensor.unsqueeze(0), SAMPLE_RATE)
        
        # Для отладки можно раскомментировать:
        # print(f"Форма вероятностей: {prob_tensor.shape}, значения: {prob_tensor}")
        
        # Получаем скалярное значение вероятности
        # Если prob_tensor имеет форму [1, 1], используем .item()
        # Если форма [1, n], нужно агрегировать (например, взять среднее)
        if prob_tensor.numel() == 1:
            speech_prob = prob_tensor.item()
        else:
            # Берем среднее по временным фреймам
            speech_prob = prob_tensor.mean().item()
    
    # Логика детектирования речи и записи
    if speech_prob > 0.5:  # Порог можно настроить (обычно 0.5)
        is_recording = True
        pause_counter = 0
        audio_buffer.append(audio_chunk.copy())
        print("🎤 Речь обнаружена", end='\r')
    elif is_recording:
        pause_counter += 1
        audio_buffer.append(audio_chunk.copy())
        
        if pause_counter >= PAUSE_CHUNKS:
            # Сохраняем записанную реплику
            if audio_buffer:
                full_audio = np.concatenate(audio_buffer, axis=0)
                timestamp = int(time.time() * 1000)
                filename = f"utterance_{timestamp}.wav"
                wavio.write(filename, full_audio, SAMPLE_RATE, sampwidth=2)
                duration = len(full_audio) / SAMPLE_RATE
                print(f"\n✅ Реплика сохранена: {filename} ({duration:.2f} сек)")

                # САМОЕ ВАЖНОЕ
                # 1. Вызов распознавания и генерации xml для tool_call
                # 2. Вызов tool_call
                # 3. Озвучивание результата.
                # TODO: переделать с глобальных параметров player и spk
                print("Распознавание реплики")
                resp = transcribe_audio(filename) # транскрибация
                print(resp)
                
                               
                tool_resp = player.tool_call(resp)
                print(f"tool_resp = {tool_resp}")

                text_to_speak1 = text_for_tts(tool_resp)
                tts_resp = spk.speak(text_to_speak1)
                print(f"tts_resp = {tts_resp}")                

                
                #############################################
                # Сбрасываем состояние для следующей реплики
                audio_buffer.clear()
                is_recording = False
                pause_counter = 0

In [22]:
# Настройка и запуск аудиопотока
print(f"Конфигурация: Частота {SAMPLE_RATE} Гц, чанк {CHUNK_SIZE} семплов")
print(f"Порог паузы: {PAUSE_THRESHOLD_MS} мс")

try:
    stream = sd.InputStream(
        channels=1,
        samplerate=SAMPLE_RATE,
        blocksize=CHUNK_SIZE,
        callback=callback,
        dtype='float32'
    )
    
    stream.start()
    print("\n🎤 Система готова. Говорите... (Ctrl+C для остановки)")
    print("Индикация: '🎤 Речь обнаружена' при активной речи")
    
    # Бесконечный цикл ожидания
    while True:
        time.sleep(0.1)
        
except KeyboardInterrupt:
    print("\n\nЗапись остановлена пользователем.")
except Exception as e:
    print(f"\nОшибка: {e}")
finally:
    if 'stream' in locals():
        stream.stop()
        stream.close()
    
    # Сохраняем последнюю реплику, если запись была прервана во время речи
    if audio_buffer and is_recording:
        full_audio = np.concatenate(audio_buffer, axis=0)
        filename = f"utterance_final_{int(time.time()*1000)}.wav"
        wavio.write(filename, full_audio, SAMPLE_RATE, sampwidth=2)
        print(f"Сохранена последняя реплика: {filename}")

Конфигурация: Частота 16000 Гц, чанк 512 семплов
Порог паузы: 500 мс

🎤 Система готова. Говорите... (Ctrl+C для остановки)
Индикация: '🎤 Речь обнаружена' при активной речи
🎤 Речь обнаружена

/home/m1/nvme/p311uns2/lib/python3.11/site-packages/wavio.py:259: ClippedDataWarning: Some data values have been clipped.  With scale=1.0, the interval of input values that will not be clipped is [-1.0000305180437934, 1.0]
  _warnings.warn(ClippedDataWarning(msg))



✅ Реплика сохранена: utterance_1770431052618.wav (1.63 сек)
Распознавание реплики
<tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_status</name><arguments></arguments></tool_call><end_of_turn>
Status of playing
tool_resp = {'status': 'success', 'message': 'Status requested'}
Отправка текста на озвучку: 'Status requested...'
Используется голос: ls_female1.wav
✅ TTS сервис доступен: Начато воспроизведение сгенерированной речи
tts_resp = {'status': 'success', 'response': {'audio_samples': 44304, 'language': None, 'message': 'Начато воспроизведение сгенерированной речи', 'speaker': 'ls_female1.wav', 'status': 'success', 'text_length': 16}}
🎤 Речь обнаружена
✅ Реплика сохранена: utterance_1770431068585.wav (3.52 сек)
Распознавание реплики
<tool_call><name>audioplay.play_request</name><arguments><artist>クリスタゲイルズ</artist><title>ララ</title><genre></genre><album></album></arguments></tool_call><end_of_turn><tool_call><n

/home/m1/nvme/p311uns2/lib/python3.11/site-packages/wavio.py:259: ClippedDataWarning: Some data values have been clipped.  With scale=1.0, the interval of input values that will not be clipped is [-1.0000305180437934, 1.0]
  _warnings.warn(ClippedDataWarning(msg))



✅ Реплика сохранена: utterance_1770432185899.wav (3.14 сек)
Распознавание реплики
<tool_call><name>audioplay.play_request</name><arguments><artist>David Ward</artist><title>Char</title></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>David Ward</artist><title>Char</title></arguments></tool_call><end_of_turn>
tool_resp = {'status': 'error', 'message': 'No matching records found'}
Отправка текста на озвучку: 'No matching records found...'
Используется голос: ls_female1.wav
✅ TTS сервис доступен: Начато воспроизведение сгенерированной речи
tts_resp = {'status': 'success', 'response': {'audio_samples': 49936, 'language': None, 'message': 'Начато воспроизведение сгенерированной речи', 'speaker': 'ls_female1.wav', 'status': 'success', 'text_length': 25}}
🎤 Речь обнаружена

...


✅ Реплика сохранена: utterance_1770433108503.wav (2.14 сек)
Распознавание реплики
<tool_call><name>audioplay.play_request</name><arguments><title>char</title></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><title>char</title></arguments></tool_call><end_of_turn>
tool_resp = {'status': 'error', 'message': 'No matching records found'}
Отправка текста на озвучку: 'No matching records found...'
Используется голос: ls_female1.wav
✅ TTS сервис доступен: Начато воспроизведение сгенерированной речи
tts_resp = {'status': 'success', 'response': {'audio_samples': 46608, 'language': None, 'message': 'Начато воспроизведение сгенерированной речи', 'speaker': 'ls_female1.wav', 'status': 'success', 'text_length': 25}}
🎤 Речь обнаружена

/home/m1/nvme/p311uns2/lib/python3.11/site-packages/wavio.py:259: ClippedDataWarning: Some data values have been clipped.  With scale=1.0, the interval of input values that will not be clipped is [-1.0000305180437934, 1.0]
  _warnings.warn(ClippedDataWarning(msg))



✅ Реплика сохранена: utterance_1770433123095.wav (2.78 сек)
Распознавание реплики
<tool_call><name>audioplay.play_request</name><arguments><artist>Barbara Streisand</artist><title>Caress My Soul</title><genre>Pop</genre><album></album></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_request</name><arguments><artist>Barbara Streisand</artist><title>Caress My Soul</title><genre>Pop</genre><album></album></arguments></tool_call><end_of_turn>
{'SourceFile': '/home/m1/music1/Nostalgie/10 - Barbara Streisand - Woman In Love.mp3', 'FileName': '10 - Barbara Streisand - Woman In Love.mp3', 'Artist': 'Barbara Streisand', 'Album': 'Nostalgie', 'Title': 'Woman In Love', 'Genre': ''}
SourceFile = /home/m1/music1/Nostalgie/10 - Barbara Streisand - Woman In Love.mp3
FileName = 10 - Barbara Streisand - Woman In Love.mp3
Artist = Barbara Streisand
Album = Nostalgie
Title = Woman In Love
Genre = 
Воспроизведение на хосте (адрес:порт): http://127.0.0.1:5000
✅ Сервис доступен: {'file'

/home/m1/nvme/p311uns2/lib/python3.11/site-packages/wavio.py:259: ClippedDataWarning: Some data values have been clipped.  With scale=1.0, the interval of input values that will not be clipped is [-1.0000305180437934, 1.0]
  _warnings.warn(ClippedDataWarning(msg))



✅ Реплика сохранена: utterance_1770433254860.wav (1.66 сек)
Распознавание реплики
<tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call><end_of_turn><tool_call><name>audioplay.play_stop</name><arguments></arguments></tool_call><end_of_turn>
Play stopping
tool_resp = {'status': 'success', 'message': 'Playback stopped'}
Отправка текста на озвучку: 'Playback stopped...'
Используется голос: ls_female1.wav
✅ TTS сервис доступен: Начато воспроизведение сгенерированной речи
tts_resp = {'status': 'success', 'response': {'audio_samples': 38928, 'language': None, 'message': 'Начато воспроизведение сгенерированной речи', 'speaker': 'ls_female1.wav', 'status': 'success', 'text_length': 16}}


Запись остановлена пользователем.
